In [1]:
import pandas as pd 
import numpy as np 
import os 
import seaborn as sns 
import matplotlib.pyplot as plt
import math
import scipy.signal as sig

In [2]:
# Set input path to run all analysis 
# folder with all raw pose data (dir_in_path) 
raw_pose_data_in_path = r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_outputs_004\subset_for_memory_c'
# fps for zeno = 30
fps = 30

output_folder = r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\test_velocity_01302025\zeno_all_height_proxy_outputs'

In [3]:
def calc_pix_size_change(start_frame, end_frame, df, fps): 
    start_row = df.loc[df['frame'] == start_frame]
    end_row = df.loc[df['frame'] == end_frame]

    # height in pixel at start and end of time group 
    pix_h_start = start_row['approx_height_Y_pix_smooth'].iloc[0]
    pix_h_end = end_row['approx_height_Y_pix_smooth'].iloc[0]

    # for consistency, only calculate when walking toward camera - height is getting bigger as walking toward 
    if pix_h_end > pix_h_start:
        # Change in pixel height from start to end 
        delta_pix_h = pix_h_end - pix_h_start

        # change in pixel height relative to start
        delta_pix_h_rel = delta_pix_h / pix_h_start

        # logarithmic scaling 
        delta_pix_h_log = math.log(pix_h_end / pix_h_start)

    else: 
        delta_pix_h = np.nan
        delta_pix_h_rel = np.nan
        delta_pix_h_log = np.nan


    return delta_pix_h, delta_pix_h_rel, delta_pix_h_log

In [5]:
# save list of all mediapipe world csv files in raw pose data folders 

raw_data_full_path_all = [] 
raw_data_file_names_all = []

depth_proxy_describe_all = pd.DataFrame()

# loop through all files in input path 
for (dir_path, dir_names, file_names) in os.walk(raw_pose_data_in_path):
    for file_name in file_names: 
        name, ext = os.path.splitext(file_name)
        ext = ext.lower()[1:]
        current_raw_data_in_path = os.path.join(dir_path, file_name) # full path to files 

        # get 
        parent_path_1, current_file_name = os.path.split(current_raw_data_in_path)
        parent_path_2, current_raw_data_folder = os.path.split(parent_path_1)
        parent_path_3, current_date = os.path.split(parent_path_2)
        parent_path_4, current_id = os.path.split(parent_path_3)
        current_id_date = os.path.join(current_id, current_date)

        # select file paths with yolo data ----------------------------------------
        if (ext == 'csv') &  ('000_raw_pose_data' in current_raw_data_in_path) & ('yolo.csv' in current_raw_data_in_path): # from run.py, outputs all saved in 000_raw_pose_data_folder
            print(current_raw_data_in_path)
            yolo_df = pd.read_csv(current_raw_data_in_path, index_col = 0)

           # pixel height -------------------------------------------------

            # calculate height in pixels - left eye to left ankle 
            l_eye_yolo_df = yolo_df.loc[yolo_df['label'] == 'left_eye']
            l_eye_yolo_df.set_index('frame', inplace=True)
            l_ankle_yolo_df = yolo_df.loc[yolo_df['label'] == 'left_ankle']
            l_ankle_yolo_df.set_index('frame', inplace=True)

            l_ankle_yolo_df = l_ankle_yolo_df.copy()
            l_ankle_yolo_df.loc[:, 'approx_height_Y_pix'] = abs(l_eye_yolo_df['Y'] - l_ankle_yolo_df['Y'])
            l_ankle_yolo_df.loc[:, 'approx_height_Y_pix_smooth'] = l_ankle_yolo_df['approx_height_Y_pix'].rolling(window=25, min_periods = 1).mean()
            
            # drop l ankle specific columns 
            height_df = l_ankle_yolo_df.drop(columns = ['X', 'Y', 'label'])
            height_df.reset_index(inplace=True)

            # add "time_group" label column, will use to group 
            height_df['time_group'] = height_df['frame'] / (fps)
            
            height_df['time_group'] = height_df['time_group'].apply(math.floor)
            # identify peaks and valleys in pixel height (approximate turn locations) 
            peaks, _ = sig.find_peaks(height_df['approx_height_Y_pix_smooth'], distance = fps)
            valleys, _ = sig.find_peaks(-height_df['approx_height_Y_pix_smooth'], distance = fps)
            peak_frames = height_df.iloc[peaks]['frame']
            valley_frames = height_df.iloc[valleys]['frame']
            peaks_valleys = np.concatenate((peak_frames, valley_frames))

#            # plot height in pixels with time groups and peaks valleys 
#             sns.scatterplot(x = 'frame', y = 'approx_height_Y_pix_smooth', data = height_df, label = 'approx_height_Y_pix_smooth')
#             plt.plot(height_df['frame'], height_df['approx_height_Y_pix_smooth'], label='Smoothed Height')
#             plt.scatter(peak_frames, height_df.iloc[peaks]['approx_height_Y_pix_smooth'], color='red', label='Peaks')
#             plt.scatter(valley_frames, height_df.iloc[valleys]['approx_height_Y_pix_smooth'], color='blue', label='Valleys')
#             for time_group in height_df['time_group'].unique(): 
#                 plt.axvline(x = time_group * (fps), color = 'grey', alpha = 0.5)
#             plt.title(current_id_date + ' ' + name)
#            plt.show()

            # calculate depth proxy values and summarize 
            depth_proxies_all = []

            for current_sec_group in height_df['time_group'].unique():
                current_height_df = height_df.loc[height_df['time_group'] == current_sec_group]

                current_start_frame = current_height_df['frame'].iloc[0]
                current_end_frame = current_height_df['frame'].iloc[-1]

                # if this current time_group group contains peak or valley, skip 
                if current_height_df.loc[current_start_frame: current_end_frame, 'frame'].isin(peaks_valleys).any():
                   # print('Skipped - contains peak or valley') 
                    delta_pix_h = np.nan
                    delta_pix_h_rel = np.nan
                    delta_pix_h_log = np.nan
                else: 
                    delta_pix_h, delta_pix_h_rel, delta_pix_h_log  = calc_pix_size_change(current_start_frame, current_end_frame, height_df, fps)
                
                # combine all time groups into one array 
                depth_proxies_all.append({'start_frame' : current_start_frame,
                                         'delta_pix_h' : delta_pix_h, 
                                         'delta_pix_h_rel' : delta_pix_h_rel,
                                         'delta_pix_h_log' : delta_pix_h_log})

                             
            # convert array to df 
            depth_proxies_all_df = pd.DataFrame(depth_proxies_all)
            depth_proxies_all_df =depth_proxies_all_df.replace([np.inf, -np.inf], np.nan) # replace inf with nan 

            depth_proxies_all_df = depth_proxies_all_df.drop(columns = ['start_frame'])    
            # convert to one row 
            depth_proxy_describe = depth_proxies_all_df.describe().unstack().to_frame().T
            depth_proxy_describe.columns = ['_'.join(col).strip() for col in depth_proxy_describe.columns]
            depth_proxy_describe = round(depth_proxy_describe, 2)
            depth_proxy_describe['id_video'] = current_id 
            depth_proxy_describe['visit_date_video'] = current_date
            depth_proxy_describe['filename'] = current_file_name
            
            # save all height and pixel proxies 
            depth_proxy_describe_all = pd.concat([depth_proxy_describe_all, depth_proxy_describe])   

C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_outputs_004\subset_for_memory_c\BW-0204\2023_07_31\000_raw_pose_data\gait_vertical_FW_1_yolo.csv
   frame  approx_height_Y_pix  approx_height_Y_pix_smooth  time_group
0      0            700.06964                  700.069640           0
1      1            697.64510                  698.857370           0
2      2            694.22046                  697.311733           0
3      3            692.88730                  696.205625           0
4      4            692.22820                  695.410140           0
C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_outputs_004\subset_for_memory_c\BW-0204\2023_07_31\000_raw_pose_data\gait_vertical_PWS_1_yolo.csv
   frame  approx_height_Y_pix  approx_height_Y_pix_smooth  time_group
0      0                  0.0                         0.0           0
1      1                  0.0                         0.0           0
2      2         

C:\Users\mmccu\AppData\Local\Temp\ipykernel_23824\802653250.py:15: RuntimeWarning: divide by zero encountered in scalar divide
  delta_pix_h_rel = delta_pix_h / pix_h_start
C:\Users\mmccu\AppData\Local\Temp\ipykernel_23824\802653250.py:18: RuntimeWarning: divide by zero encountered in scalar divide
  delta_pix_h_log = math.log(pix_h_end / pix_h_start)


C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_outputs_004\subset_for_memory_c\BW-0205\2023_08_02\000_raw_pose_data\gait_vertical_FW_1_yolo.csv
   frame  approx_height_Y_pix  approx_height_Y_pix_smooth  time_group
0      0            231.08405                  231.084050           0
1      1            240.68867                  235.886360           0
2      2            246.27069                  239.347803           0
3      3            248.56501                  241.652105           0
4      4            250.62430                  243.446544           0
C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_outputs_004\subset_for_memory_c\BW-0205\2024_08_07\000_raw_pose_data\gait_vertical_FW_1_yolo.csv
   frame  approx_height_Y_pix  approx_height_Y_pix_smooth  time_group
0      0                  0.0                         0.0           0
1      1                  0.0                         0.0           0
2      2          

C:\Users\mmccu\AppData\Local\Temp\ipykernel_23824\802653250.py:15: RuntimeWarning: divide by zero encountered in scalar divide
  delta_pix_h_rel = delta_pix_h / pix_h_start
C:\Users\mmccu\AppData\Local\Temp\ipykernel_23824\802653250.py:18: RuntimeWarning: divide by zero encountered in scalar divide
  delta_pix_h_log = math.log(pix_h_end / pix_h_start)


   frame  approx_height_Y_pix  approx_height_Y_pix_smooth  time_group
0      0            592.44890                  592.448900           0
1      1            594.19965                  593.324275           0
2      2            595.92865                  594.192400           0
3      3            595.14386                  594.430265           0
4      4            591.72070                  593.888352           0
C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_outputs_004\subset_for_memory_c\BW-0209\2023_07_28\000_raw_pose_data\gait_vertical_FW_1_yolo.csv
   frame  approx_height_Y_pix  approx_height_Y_pix_smooth  time_group
0      0            185.75241                  185.752410           0
1      1            186.66654                  186.209475           0
2      2            185.91600                  186.111650           0
3      3            183.24942                  185.396093           0
4      4            178.82908                  184.082

...

C:\Users\mmccu\AppData\Local\Temp\ipykernel_23824\802653250.py:15: RuntimeWarning: divide by zero encountered in scalar divide
  delta_pix_h_rel = delta_pix_h / pix_h_start
C:\Users\mmccu\AppData\Local\Temp\ipykernel_23824\802653250.py:18: RuntimeWarning: divide by zero encountered in scalar divide
  delta_pix_h_log = math.log(pix_h_end / pix_h_start)


C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_outputs_004\subset_for_memory_c\BW-0357\2024_09_25\000_raw_pose_data\gait_vertical_FW_1_yolo.csv
   frame  approx_height_Y_pix  approx_height_Y_pix_smooth  time_group
0      0                  0.0                         0.0           0
1      1                  0.0                         0.0           0
2      2                  0.0                         0.0           0
3      3                  0.0                         0.0           0
4      4                  0.0                         0.0           0
C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_outputs_004\subset_for_memory_c\BW-0357\2024_09_25\000_raw_pose_data\gait_vertical_PWS_1_yolo.csv


C:\Users\mmccu\AppData\Local\Temp\ipykernel_23824\802653250.py:15: RuntimeWarning: divide by zero encountered in scalar divide
  delta_pix_h_rel = delta_pix_h / pix_h_start
C:\Users\mmccu\AppData\Local\Temp\ipykernel_23824\802653250.py:18: RuntimeWarning: divide by zero encountered in scalar divide
  delta_pix_h_log = math.log(pix_h_end / pix_h_start)


   frame  approx_height_Y_pix  approx_height_Y_pix_smooth  time_group
0      0                  0.0                         0.0           0
1      1                  0.0                         0.0           0
2      2                  0.0                         0.0           0
3      3                  0.0                         0.0           0
4      4                  0.0                         0.0           0


C:\Users\mmccu\AppData\Local\Temp\ipykernel_23824\802653250.py:15: RuntimeWarning: divide by zero encountered in scalar divide
  delta_pix_h_rel = delta_pix_h / pix_h_start
C:\Users\mmccu\AppData\Local\Temp\ipykernel_23824\802653250.py:18: RuntimeWarning: divide by zero encountered in scalar divide
  delta_pix_h_log = math.log(pix_h_end / pix_h_start)


In [6]:
depth_proxy_describe_all

,delta_pix_h_count,delta_pix_h_mean,delta_pix_h_std,delta_pix_h_min,delta_pix_h_25%,delta_pix_h_50%,delta_pix_h_75%,delta_pix_h_max,delta_pix_h_rel_count,delta_pix_h_rel_mean,...,delta_pix_h_log_mean,delta_pix_h_log_std,delta_pix_h_log_min,delta_pix_h_log_25%,delta_pix_h_log_50%,delta_pix_h_log_75%,delta_pix_h_log_max,id_video,visit_date_video,filename
0,7.0,55.34,17.39,33.20,43.28,52.44,68.29,78.63,7.0,0.20,...,0.18,0.02,0.14,0.17,0.19,0.20,0.20,BW-0204,2023_07_31,gait_vertical_FW_1_yolo.csv
0,11.0,136.86,186.63,23.45,43.42,62.42,73.63,521.14,9.0,0.18,...,0.16,0.03,0.10,0.15,0.16,0.17,0.21,BW-0204,2023_07_31,gait_vertical_PWS_1_yolo.csv
0,2.0,77.08,36.02,51.61,64.34,77.08,89.81,102.55,2.0,0.34,...,0.29,0.12,0.20,0.24,0.29,0.33,0.37,BW-0205,2023_08_02,gait_vertical_FW_1_yolo.csv
0,6.0,98.56,51.47,35.29,65.24,89.14,136.20,168.66,6.0,0.36,...,0.30,0.11,0.17,0.21,0.35,0.38,0.41,BW-0205,2024_08_07,gait_vertical_FW_1_yolo.csv
0,9.0,79.31,66.06,6.41,33.11,52.73,115.38,212.49,9.0,0.27,...,0.23,0.13,0.03,0.16,0.21,0.35,0.45,BW-0205,2024_08_07,gait_vertical_PWS_1_yolo.csv


In [7]:
depth_proxy_describe_all.to_csv(os.path.join(output_folder, 'depth_proxy_describe_c.csv')) 